<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Raw_data/Accessing_Steam_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install steamspypi

In [ ]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests
import steamspypi
from sklearn.utils import shuffle

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)
pd.set_option('display.max_rows', None)

In [ ]:
#Get all game ids and names
#URL call found here: https://partner.steamgames.com/doc/webapi/ISteamApps
URL = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'

response = requests.get(url=URL)
json_data = response.json()
GameIDs = pd.DataFrame.from_dict(json_data['applist']['apps'])
#Clean up the dataframe to remove empty strings and test/demo games
GameIDs['name'] = GameIDs['name'].str.strip()
GameIDs['name'] = GameIDs['name'].str.lower()
GameIDs = GameIDs[GameIDs['name'].isin(['','pieterw test app76 ( 216938 )','test2','test3', 'tidewoken public test', 
                                        'now testing: 407', 'test re(quietmansion1 special teaser)', '<h1>test</h1>', 
                                        'test', 'test project', 'steamvr performance test', 'testcontent', 'vrq test'
                                        ]) == False]
GameIDs = GameIDs[GameIDs['name'].str.contains('playtest')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('closed testing')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('testapp')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains(' test ')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('betatest')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('test server')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('beta test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('tidewoken public test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('open test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('dev test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('- test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('feature test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('technical test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('early access testing')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('_test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains(' demo')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('public test')==False]


In [ ]:
GameIDs.shape

(125729, 2)

In [ ]:
#Create function to collect data from APIs
def CollectSteamData(GameIDDF):
    '''
    input: dataframe containing IDs and names of games 
    output: dataframe containing all api data from a random sample of the games
    '''
    #Steam API 1: primary game data
    #https://stackoverflow.com/questions/69512319/steam-api-to-get-game-info
    #Steam API 2: Review data
    #https://partner.steamgames.com/doc/store/getreviews
    #Steamspy API: Supplemental usage and cost data
    # https://pypi.org/project/steamspypi/
    # https://steamspy.com/api.php
    
    #Randomize the data frame
    IDs = shuffle(GameIDDF)
    GameDict = {}
    starttime = time.time()
    for appid in IDs['appid']:
        try:
            gameURL = 'http://store.steampowered.com/api/appdetails?appids=' + str(appid)
            response = requests.get(url=gameURL)
            json_data = response.json()
            GameData = json_data[str(appid)]['data']
            time.sleep(1) # 1 second rate limit on API calls
            reviewURL = 'http://store.steampowered.com/appreviews/' + str(appid) + '?json=1'
            response = requests.get(url=reviewURL)
            json_data = response.json()
            ReviewScore = json_data['query_summary']['review_score']
            ReviewScoreDesc = json_data['query_summary']['review_score_desc']
            reviewText = ''
            for review in json_data['reviews']:
                reviewText = reviewText + review['review']
            
            ReviewDict = {'Review Score':ReviewScore, 'Review Score Description': ReviewScoreDesc, 'Top Reviews by Upvotes':reviewText}

            data_request = dict()
            data_request['request'] = 'appdetails'
            data_request['appid'] = str(appid)
            steamspydata = steamspypi.download(data_request)

            # Combine all three json dictionaries and convert to dataframe
            GameData.update(ReviewDict)
            GameData.update(steamspydata)
            time.sleep(1) # 1 second rate limit on API calls

        except: #games that do not have any associated data or other failed api calls
            time.sleep(1)
        endtime = time.time()
        elapsedtime = (endtime-starttime)/60
        if elapsedtime >= 480: #If Greater than or equal to 8 hours, then end
            break
        #add all data for current app loop to GameDict
        GameDict.update({str(appid): GameData})
    #Convert to Dataframe
    GameDF = pd.DataFrame.from_dict(GameDict, orient='index')

    return GameDF

In [ ]:
Sample_Game_Data = CollectSteamData(GameIDs)

In [ ]:
from google.colab import files
Sample_Game_Data.to_csv('RawSteamGameData.csv') 
files.download('RawSteamGameData.csv')

"A message to the future generations. Don't let this game die :luv:\n---{ Graphics }---\n☑ You forget what reality is\n☐ Beautiful\n☐ Good\n☐ Decent\n☐ Bad\n☐ Don‘t look too long at it\n☐ MS-DOS\n\n---{ Gameplay }---\n☑ Very good\n☐ Good\n☐ It's just gameplay\n☐ Mehh\n☐ Watch paint dry instead\n☐ Just don't\n\n---{ Audio }---\n☑ Eargasm\n☐ Very good\n☐ Good\n☐ Not too bad\n☐ Bad\n☐ I'm now deaf\n\n---{ Audience }---\n☐ Kids\n☑ Teens\n☑ Adults\n☑ Grandma\n\n---{ PC Requirements }---\n☑ Check if you can run paint\n☐ Potato\n☐ Decent\n☐ Fast\n☐ Rich boi\n☐ Ask NASA if they have a spare computer\n\n---{ Difficulty }---\n☐ Just press 'W'\n☐ Easy\n☐ Easy to learn / Hard to master\n☑ Significant brain usage\n☐ Difficult\n☐ Dark Souls\n\n---{ Grind }---\n☐ Nothing to grind\n☐ Only if u care about leaderboards/ranks\n☐ Isn't necessary to progress\n☐ Average grind level\n☐ Too much grind\n☑ You'll need a second live for grinding\n\n---{ Story }---\n☑ No Story\n☐ Some lore\n☐ Average\n☐ Good\n☐ L

In [ ]:


data_request = dict()
data_request['request'] = 'appdetails'
data_request['appid'] = '730'

data = steamspypi.download(data_request)

In [ ]:
IDs = ['10', '20', '30']
GameDict = {}
starttime = time.time()
for appid in IDs:
    try:
        gameURL = 'http://store.steampowered.com/api/appdetails?appids=' + str(appid)
        response = requests.get(url=gameURL)
        json_data = response.json()
        GameData = json_data[str(appid)]['data']
        time.sleep(1) # 1 second rate limit on API calls
        
        reviewURL = 'http://store.steampowered.com/appreviews/' + str(appid) + '?json=1'
        response = requests.get(url=reviewURL)
        json_data = response.json()
        ReviewScore = json_data['query_summary']['review_score']
        ReviewScoreDesc = json_data['query_summary']['review_score_desc']
        reviewText = ''
        for review in json_data['reviews']:
            reviewText = reviewText + review['review']
        
        ReviewDict = {'Review Score':ReviewScore, 'Review Score Description': ReviewScoreDesc, 'Top Reviews by Upvotes':reviewText}

        data_request = dict()
        data_request['request'] = 'appdetails'
        data_request['appid'] = str(appid)
        steamspydata = steamspypi.download(data_request)

        # # Combine all three json dictionaries and convert to dataframe
        GameData.update(ReviewDict)
        GameData.update(steamspydata)
        time.sleep(1) # 1 second rate limit on API calls
    except: #games that do not have any associated data or other failed api calls
        time.sleep(1)
        print(appid)

    #add all data for current app loop to GameDict
    GameDict.update({str(appid): GameData})
    endtime = time.time()
    elapsedtime = (endtime-starttime)/60
    print(elapsedtime)
    if elapsedtime >= 480: #If Greater than or equal to 8 hours, then end
        break

0.04852695067723592
0.09534444411595662
0.14404764175415039


In [ ]:
GameDict

{'10': {'Review Score': 9,
  'Review Score Description': 'Overwhelmingly Positive',
  'Top Reviews by Upvotes': "A message to the future generations. Don't let this game die :luv:\n---{ Graphics }---\n☑ You forget what reality is\n☐ Beautiful\n☐ Good\n☐ Decent\n☐ Bad\n☐ Don‘t look too long at it\n☐ MS-DOS\n\n---{ Gameplay }---\n☑ Very good\n☐ Good\n☐ It's just gameplay\n☐ Mehh\n☐ Watch paint dry instead\n☐ Just don't\n\n---{ Audio }---\n☑ Eargasm\n☐ Very good\n☐ Good\n☐ Not too bad\n☐ Bad\n☐ I'm now deaf\n\n---{ Audience }---\n☐ Kids\n☑ Teens\n☑ Adults\n☑ Grandma\n\n---{ PC Requirements }---\n☑ Check if you can run paint\n☐ Potato\n☐ Decent\n☐ Fast\n☐ Rich boi\n☐ Ask NASA if they have a spare computer\n\n---{ Difficulty }---\n☐ Just press 'W'\n☐ Easy\n☐ Easy to learn / Hard to master\n☑ Significant brain usage\n☐ Difficult\n☐ Dark Souls\n\n---{ Grind }---\n☐ Nothing to grind\n☐ Only if u care about leaderboards/ranks\n☐ Isn't necessary to progress\n☐ Average grind level\n☐ Too much gri

In [ ]:
GameDF = pd.DataFrame.from_dict(GameDict, orient='index')

In [ ]:
GameDF

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
10,game,Counter-Strike,10,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,None,{'minimum': ' <p><strong>Minimum:</strong>...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[Valve],[Valve],"{'currency': 'USD', 'initial': 999, 'final': 9...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 118774},"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",9,Overwhelmingly Positive,A message to the future generations. Don't let...,10,Valve,Valve,,193430,4955,0,"10,000,000 .. 20,000,000",9661,110,186,32,999,999,0,14752,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5379, 'FPS': 4801, 'Multiplayer': 3..."
20,game,Team Fortress Classic,20,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,None,{'minimum': ' <p><strong>Minimum:</strong>...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[Valve],[Valve],"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4489},"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",8,Very Positive,"It's like Team Fortress 2, but with no aimbots...",20,Valve,Valve,,5420,897,0,"5,000,000 .. 10,000,000",226,0,20,0,499,499,0,80,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 257..."
30,game,Day of Defeat,30,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': ' <p><strong>Minimum:</strong>...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",[Valve],[Valve],"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 3136},"{'